In [1]:
import pickle

# Replace 'your_file_path.pkl' with the path to your .pkl file
file_path = 'splitDocuments.pkl'

with open(file_path, 'rb') as file:
    contents = pickle.load(file)

# Extracting page_content from each document object
docs_texts = []
for sublist in contents:
    for doc in sublist:  # Assuming each element in sublist is a document object
        docs_texts.append(doc.page_content)

In [2]:
import torch
from sentence_transformers import SentenceTransformer, util
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("Salesforce/SFR-Embedding-Mistral", device=device)

/home/trevea/miniconda3/envs/nlp-rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [01:58<00:00, 39.45s/it]


In [45]:
import torch
from sentence_transformers import SentenceTransformer
from torch.nn import DataParallel
import numpy as np

class MistralEmbedder:
    def __init__(self, model_name="Salesforce/SFR-Embedding-Mistral", device='cuda'):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(model_name, device=device)
    
    def embed_documents(self, texts):
        """
        Generate embeddings for a list of text documents using the Mistral model.
        
        Parameters:
        - texts (List[str]): A list of texts to embed.
        
        Returns:
        - List of embeddings as numpy arrays.
        """
        # Ensure texts is a list for batch processing
        if isinstance(texts, str):
            texts = [texts]
        
        # Generate embeddings and return them as numpy arrays
        embeddings = self.model.encode(texts)
        return embeddings
    def embed_query(self, query):
        """
        Generate an embedding for a single query string using the Mistral model.
        
        Parameters:
        - query (str): The query string to embed.
        
        Returns:
        - A numpy array representing the embedding of the query.
        """
        # Generate the embedding for the query
        embedding = self.model.encode(query, convert_to_numpy=True)

        # Convert numpy array to list if necessary
        embedding_list = embedding.tolist() if isinstance(embedding, np.ndarray) else embedding

        # Return the embedding
        return embedding_list

# Initialize the embedding wrapper
mistral_embedder = MistralEmbedder()

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.21it/s]


In [27]:
mistral_embedder.embed_query("Hello")

array([ 5.433129 , -0.4557455, -1.6711234, ...,  4.778883 , -2.628141 ,
        2.1035042], dtype=float32)

In [44]:
import torch

# Set the current device to CUDA device 1
torch.cuda.set_device(1)

# Assuming mistral_embedder is your instance of MistralEmbedder
# Delete the model attribute from your instance
del mistral_embedder.model

# Clear the CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Optionally, check the memory status
print(torch.cuda.memory_summary(device=1, abbreviated=True))

AttributeError: 'MistralEmbedder' object has no attribute 'model'

In [3]:
from langchain_community.llms import LlamaCpp


n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q6_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32

In [4]:
from langchain_community.vectorstores import Chroma

# Initialize all_texts with leaf_texts
all_texts = docs_texts.copy()

# Build the vector store with Chroma
vectorstore = Chroma.from_texts(texts=all_texts, embedding=mistral_embedder, persist_directory="MistralEmbed/")
retriever = vectorstore.as_retriever()

In [46]:
vectorstore._embedding_function = mistral_embedder

In [47]:
query = "What is LTI"
docs = vectorstore.similarity_search(query)

In [48]:
print(docs)

[Document(page_content="Learn at LTI The LTI's degree programs draw from a common set of courses and core skills, but emphasize different types of expertise that prepare you for a wide range of career options. All of our programs provide the hands-on experience and rigorous curriculum that are the hallmark of computer science at Carnegie Mellon. Program Type: Ph.D. Programs Program Name: Ph.D. in Language and Information Technology Overview: The Ph.D. in LTI focuses on developing the next generation of scientific and"), Document(page_content="Learn at LTI The LTI's degree programs draw from a common set of courses and core skills, but emphasize different types of expertise that prepare you for a wide range of career options. All of our programs provide the hands-on experience and rigorous curriculum that are the hallmark of computer science at Carnegie Mellon. Program Type: Ph.D. Programs Program Name: Dual-Degree Ph.D. in Language and Information Technologies (Portugal Partnership) Ov

In [49]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnablePick
from langchain_core.prompts.chat import HumanMessagePromptTemplate, PromptTemplate

# rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt.messages
# prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use one sentence maximum and keep the answer CONCISE. Keep the answer CONCISE.\nQuestion: {question} \nContext: {context} \nAnswer:"))
# rag_prompt.messages = [prompt]

In [50]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [51]:
import os
from tqdm import tqdm


# Load questions from questions.txt
with open("SubmissionData/test/questions.txt", "r") as f:
    questions = [line.strip() for line in f.readlines()]

# Output directory for answers
output_dir = "SubmissionData/system_outputs/"
answer_file = os.path.join(output_dir, "MistralEmbed.txt")

# Run the question-answering loop and save answers
answers = []
with tqdm(total=len(questions), desc="Answering questions") as progress_bar:
    with open(answer_file, "w") as f:
        for question in questions:
            response = qa_chain.invoke(question)
            f.write(response.replace("\n",""), "\n")
            answers.append(response)
            progress_bar.update(1)

Answering questions:   0%|          | 0/79 [00:00<?, ?it/s]
llama_print_timings:        load time =    2809.91 ms
llama_print_timings:      sample time =      27.84 ms /    78 runs   (    0.36 ms per token,  2801.42 tokens per second)
llama_print_timings: prompt eval time =    2809.75 ms /   107 tokens (   26.26 ms per token,    38.08 tokens per second)
llama_print_timings:        eval time =    5852.29 ms /    77 runs   (   76.00 ms per token,    13.16 tokens per second)
llama_print_timings:       total time =    8933.81 ms /   184 tokens
Answering questions:   1%|▏         | 1/79 [00:09<11:42,  9.01s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    2809.91 ms
llama_print_timings:      sample time =       9.60 ms /    26 runs   (    0.37 ms per token,  2707.49 tokens per second)
llama_print_timings: prompt eval time =    1671.30 ms /    62 tokens (   26.96 ms per token,    37.10 tokens per second)
llama_print_timings:        eval time =    1697.83 ms /  